## Regression

In this script, we perform regression on the cooling load and the result is evaluated statistically. We compare a baseline, a linear regression model and an ANN model.

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [3]:
import warnings
warnings.filterwarnings('ignore')

#### Read/prepare data

In [4]:
datapath = "data/"
filename = 'ENB2012_data.csv'
df = pd.read_csv(datapath+filename)
col_names = ['relative_compactness', 'surface_area', 'wall_area', 'roof_area', 'overall_height', 'orientation', 'glazing_area', 'glazing_area_distribution', 'heating_load', 'cooling_load']
df.columns = col_names
display(df)
display(df.describe())

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.50,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.50,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.50,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.50,3.5,4,0.4,5,16.48,16.61


,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307201,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090196,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


In [16]:
# Set target and data
y = df['cooling_load']

X = df.iloc[: , :8]
X = pd.DataFrame(StandardScaler().fit_transform(X))
X.columns = col_names[:8]

### Compare models: Two-level (nested) cross-validation

For baseline: Compute the largest class on the training data, and predict everything in the test data as belonging to that class. 
<br/>-> corresponding to logistic regression with bias term and no features.

For logistic regression: Inner fold is estimating lambda, the complexity controlling parameter (called C in sklearn)

For KNN: Inner fold is estimating K, the number of neighbours in the algorithm


In [17]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
import torch
from toolbox_02450 import train_neural_net
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV

In [18]:
# Number of folds in inner and outer cross validation
K1, K2 = 10, 10

# Parameter grids. I.e. what vals to check for optimality in respective model (for inner folds)
p_grid_reg = np.power(10., range(-9, 2))
p_grid_ann = [32, 64, 128, 256, 512, 1024]

In [44]:
res = {"outer_fold": [], "reg_lambda_i": [], "reg_test_error_i": [], "ANN_nb_i": [],
       "ANN_test_error_i": [], "baseline_test_error_i": []}
res["outer_fold"] = range(K1)

# Define variables for the ANN model
loss_fn = torch.nn.MSELoss()
max_iter = 10000
N, M = X.shape

# K-fold CrossValidation with two layers
CV = model_selection.KFold(K1, shuffle=True)
for k, (train_index, test_index) in enumerate(CV.split(X, y)):
    X_train = X.iloc[train_index, :]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index, :]
    y_test = y.iloc[test_index]

    ### Baseline estimation error ###
    dummy = DummyRegressor(strategy='mean')
    dummy.fit(X_train, y_train)
    res['baseline_test_error_i'].append(
        mean_squared_error(dummy.predict(X_test), y_test))

    cv_inner = model_selection.KFold(K2, shuffle=True)

    ### Ridge regression ###
    ridge_model = RidgeCV(alphas=p_grid_reg, cv=cv_inner,
                          scoring='neg_mean_absolute_error').fit(X_train, y_train)
    best_param_reg = ridge_model.alpha_
    y_pred_reg = ridge_model.predict(X_test)
    err_reg = mean_squared_error(y_test, y_pred_reg)
    res["reg_lambda_i"].append(best_param_reg)
    res["reg_test_error_i"].append(err_reg)

    ### ANN inner cross validation ###
    # Initializing test error matrix
    S = len(p_grid_ann)
    Error_test = np.zeros((S, K2))
    # K-fold cross-validation for model selection
    for k, (train_index_inner, test_index_inner) in enumerate(cv_inner.split(X_train, y_train)):
        print('\nCrossvalidation fold: {0}/{1}'.format(k+1, K2))

        # Extract training and test set for current CV fold,
        # and convert them to PyTorch tensors
        X_train_inner = torch.Tensor(X_train.iloc[train_index_inner, :].values)
        y_train_inner = torch.Tensor([[i] for i in y_train.iloc[train_index_inner]])
        X_test_inner = torch.Tensor(X_train.iloc[test_index_inner, :].values)
        y_test_inner = torch.Tensor([[i] for i in y_train.iloc[test_index_inner]])

        # Compute the error for each number of hidden unit
        for i, n_hidden_units in enumerate(p_grid_ann):

            def model(): return torch.nn.Sequential(
                # M features to H hiden units
                torch.nn.Linear(M, n_hidden_units),
                # 1st transfer function, either Tanh or ReLU:
                torch.nn.Tanh(),  # torch.nn.ReLU(),
                # H hidden units to 1 output neuron
                torch.nn.Linear(n_hidden_units, 1)
            )

            net, final_loss, learning_curve = train_neural_net(model,
                                                               loss_fn,
                                                               X=X_train_inner,
                                                               y=y_train_inner,
                                                               n_replicates=1,
                                                               max_iter=max_iter)

            y_test_pred_ann = net(torch.Tensor(X_test_inner))
            err_ann_inner = loss_fn(y_test_inner, y_test_pred_ann)
            Error_test[i, k] = err_ann_inner

    # Find the best number of hidden unit
    generalization_error = Error_test.mean(1)
    best_n_hidden_units = p_grid_ann[np.argmin(generalization_error)]

    print(
        f'\n\tBest loss error: {err_ann_inner} for {best_n_hidden_units} number of hidden units\n')

    # Compute the final error
    def model(): return torch.nn.Sequential(
        # M features to H hiden units
        torch.nn.Linear(M, best_n_hidden_units),
        # 1st transfer function, either Tanh or ReLU:
        torch.nn.Tanh(),  # torch.nn.ReLU(),
        # H hidden units to 1 output neuron
        torch.nn.Linear(best_n_hidden_units, 1)
    )
    net, final_loss, learning_curve = train_neural_net(model,
                                                       loss_fn,
                                                       X=torch.Tensor(
                                                           X_train.values),
                                                       y=torch.Tensor(
                                                           [[i] for i in y_train]),
                                                       n_replicates=1,
                                                       max_iter=max_iter)
    y_pred_ann = net(torch.Tensor(X_test.values))
    err_ann = loss_fn(torch.Tensor([[i] for i in y_test]), y_pred_ann)
    print('\n\tBest loss final_loss: {}\n'.format(err_ann))

    res["ANN_nb_i"].append(best_n_hidden_units)
    res["ANN_test_error_i"].append(float(err_ann))



Crossvalidation fold: 1/10

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	20.879734	0.0023197196
		2000	9.926063	0.00017732818
		3000	8.311753	0.00019386971
		4000	6.4745326	0.00037399252
		5000	4.2660475	0.0003909476
		6000	3.1269403	0.00024743524
		7000	2.5754662	0.00015281464
		8000	2.2216766	6.0521834e-05
		9000	2.1297555	4.5112385e-05
		10000	1.8142205	0.0013363606
		Final loss:
		10000	1.8142205	0.0013363606

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	10.318162	0.0004571186
		2000	8.210988	0.0001999636
		3000	6.032011	0.00043933105
		4000	3.659596	0.0006886052
		5000	2.0668273	0.0004502579
		6000	1.4244848	0.00028896757
		7000	1.1478697	0.00017745265
		8000	0.92761034	0.00029163773
		9000	0.68713784	0.00029527393
		10000	0.5101306	0.00037095233
		Final loss:
		10000	0.5101306	0.00037095233

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	9.542018	0.00013300873
		2000	7.430742	0.0003440941
		3000	4.4050155	0.0006217185
		4000	2.397431	0.0005890783
		5000	1.3013462	0.000520043

In [45]:
display(pd.DataFrame.from_dict(data=res))

,outer_fold,reg_lambda_i,reg_test_error_i,ANN_nb_i,ANN_test_error_i,baseline_test_error_i
0,0,0.10,8.484288,256,0.676693,76.638126
1,1,0.10,9.597150,128,0.485322,90.248809
2,2,0.01,12.432867,128,0.798719,93.629979
3,3,0.10,15.093149,128,0.572757,102.308845
4,4,0.10,9.958477,128,0.727756,107.844675
5,5,0.10,8.523682,128,0.592399,70.230489
6,6,0.10,12.025163,512,0.292000,81.191039
7,7,0.10,9.894183,128,0.466450,102.685261
8,8,0.10,7.893860,256,0.421557,85.439852
9,9,0.10,10.830335,256,0.493364,96.222110


In [44]:
from sklearn.linear_model import Ridge
from toolbox_02450 import *
loss = 2
K = 10
m = 1
J = 0
r_baseline_ann = []
r_baseline_linear = []
r_linear_ann = []
kf = model_selection.KFold(n_splits=K, shuffle=True)



for dm in range(m):
    y_true = []

    # for train_index, test_index in kf.split(X):
    for train_index, test_index in kf.split(X, y):
        X_train, y_train = X.iloc[train_index,:], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index, :], y.iloc[test_index]

        # Test the ANN model
        n_hidden_units = 128
        loss_fn = torch.nn.MSELoss()
        max_iter = 10000
        N, M = X.shape
        def model(): return torch.nn.Sequential(
            # M features to H hiden units
            torch.nn.Linear(M, n_hidden_units),
            # 1st transfer function, either Tanh or ReLU:
            torch.nn.Tanh(),  # torch.nn.ReLU(),
            # H hidden units to 1 output neuron
            torch.nn.Linear(n_hidden_units, 1)
        )
        net, final_loss, learning_curve = train_neural_net(model,
                                                        loss_fn,
                                                        X=torch.Tensor(
                                                            X_train.values),
                                                        y=torch.Tensor(
                                                            [[i] for i in y_train]),
                                                        n_replicates=1,
                                                        max_iter=max_iter)
        y_pred_ann = np.array([elmt[0] for elmt in net(torch.Tensor(X_test.values)).detach().numpy()])

        # Test the linear regression
        lmbda = 0.1
        ridge = Ridge(lmbda)
        ridge.fit(X_train,y_train)
        y_pred_linear = ridge.predict(X_test)


        # Test the baseline mosel
        dummy = DummyRegressor(strategy='mean')
        dummy.fit(X_train, y_train)
        y_pred_baseline = dummy.predict(X_test)

        y_true.append(y_test)

        r_baseline_ann.append( np.mean( np.abs( y_pred_baseline-y_test ) ** loss - np.abs( y_pred_ann-y_test) ** loss ) )
        r_baseline_linear.append( np.mean( np.abs( y_pred_baseline-y_test ) ** loss - np.abs( y_pred_linear-y_test) ** loss ) )
        r_linear_ann.append( np.mean( np.abs( y_pred_linear-y_test ) ** loss - np.abs( y_pred_ann-y_test) ** loss ) )


	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	9.574192	0.00018862358
		2000	7.0304513	0.0004136264
		3000	4.1443114	0.000583235
		4000	2.2464445	0.0006995538
		5000	1.3295815	0.00045185772
		6000	0.8499211	0.0005654158
		7000	0.44858631	0.00059783296
		8000	0.27224034	0.00045748733
		9000	0.17906933	0.00035270496
		Final loss:
		9620	0.14812699	4.0238865e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	9.357733	0.00020724801
		2000	7.166668	0.00043235967
		3000	4.0874314	0.000646458
		4000	2.1059375	0.00062838744
		5000	1.2711159	0.00035418736
		6000	0.85538805	0.0004205594
		7000	0.5393388	0.0004336909
		8000	0.32510364	0.00045008145
		9000	0.22060616	0.00040423707
		10000	0.15920685	0.0002582589
		Final loss:
		10000	0.15920685	0.0002582589

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	9.31797	0.0002242962
		2000	6.4896226	0.0005000538
		3000	3.4807093	0.00065549853
		4000	1.9121776	0.0005399032
		5000	1.0818312	0.00045940024
		6000	0.6559439	0.00042916945
		7000	0.42949003	0.

In [45]:
# Initialize parameters and run test appropriate for setup II
alpha = 0.05
rho = 1/K
p_setupII, CI_setupII = correlated_ttest(r_baseline_ann, rho, alpha=alpha)
print("Baseline vs. ANN")
print("CI_setupII:", CI_setupII)
print("p_setupII:", p_setupII)
p_setupII, CI_setupII = correlated_ttest(r_baseline_linear, rho, alpha=alpha)
print("Baseline vs. Linear")
print("CI_setupII:", CI_setupII)
print("p_setupII:", p_setupII)
p_setupII, CI_setupII = correlated_ttest(r_linear_ann, rho, alpha=alpha)
print("Linear vs. ANN")
print("CI_setupII:", CI_setupII)
print("p_setupII:", p_setupII)

Baseline vs. ANN
CI_setupII: (80.59107618090246, 99.15977604935301)
p_setupII: 4.07493892317395e-09
Baseline vs. Linear
CI_setupII: (72.64256297300473, 87.87192888720192)
p_setupII: 1.9176239991558108e-09
Linear vs. ANN
CI_setupII: (7.649425755198767, 11.586934614850069)
p_setupII: 1.5476454023546472e-06
